<a href="https://colab.research.google.com/github/smartwatch11/ML/blob/main/HSE_HW2_EgorRybin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the required library for data processing, works with CSV file
import pandas as pd

# Reading the dataset
df = pd.read_csv('books_train.csv', error_bad_lines=False)
# Rename column with bad spaces
df.rename(columns={"  num_pages":"num_pages"}, inplace=True)


# Unify the langauge codes
encoding = {'language_code':{'en-US': 'eng', 'en-GB': 'eng', 'en-CA': 'eng'}}
df.replace(encoding, inplace=True)
# Encode categorical features as an integer array
enc = OrdinalEncoder()
enc.fit(df[['language_code']])
# Apply ordinal encoding on language_code to convert it into numerical column
df[['language_code']] = enc.fit_transform(df[['language_code']]) 

# Convert data type of publication_date from object into date type
df['publication_date'] = pd.to_datetime(df['publication_date'], format='%m/%d/%Y', errors='coerce')
# Extract year of publication in a separate column
df['year'] = pd.DatetimeIndex(df['publication_date']).year
# Add a new feature which has the number of occurences of each book
df['num_occ'] = df.groupby('title')['title'].transform('count') 

# creating the future Y_TRAIN massive with average_rating and deleting non-numerical features
label = df['average_rating'].values
df.drop(['bookID', 'title', 'authors', 'isbn', 'isbn13', 'publication_date', 'publisher', 'average_rating'], axis=1, inplace=True)

# Split the Data into 70% - 30%
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, label, test_size=0.3)

# after that I studied many various code (models) for this dataset on the Internet
# and just tryied to chouse the best model, this turned out to be RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

parameters = {
    'n_estimators': [50, 100, 150, 200],
    'max_depth': [3, 5, 7, 10, 12, 15],
    'min_samples_split': [5, 10, 15],
    'min_samples_leaf': [5, 10, 15]
}
#cv - Determines the cross-validation splitting strategy.
grad_rf = GridSearchCV(model, parameters, refit=True, cv=10)
# Train the model
grad_rf.fit(X_train, y_train)

# reading test dataset
dft = pd.read_csv('books_test.csv', error_bad_lines=False)

# do the same manipulations as with the training dataset
dft.rename(columns={"  num_pages":"num_pages"}, inplace=True)
encoding1 = {'language_code':{'en-US': 'eng', 'en-GB': 'eng', 'en-CA': 'eng'}} # Unify the langauge codes
dft.replace(encoding1, inplace=True)
enc1 = OrdinalEncoder()
enc1.fit(dft[['language_code']])
dft[['language_code']] = enc1.fit_transform(dft[['language_code']]) 
dft['publication_date'] = pd.to_datetime(dft['publication_date'], format='%m/%d/%Y', errors='coerce') 
dft['year'] = pd.DatetimeIndex(dft['publication_date']).year 
dft['num_occ'] = dft.groupby('title')['title'].transform('count')

# save bookID in separate array, so that after writing it to a file
bookID = dft['bookID'].values
dft.drop(['bookID', 'title', 'authors', 'isbn', 'isbn13', 'publication_date', 'publisher'], axis=1, inplace=True)

#predict average_rating for test dataset on the trained model
test_ratings = grad_rf.predict(dft)

# and write it to a file
with open('file.txt', 'w') as fw:
    fw.write('bookID,average_rating'+'\n')
    for i in range(len(bookID)):
        fw.write(str(bookID[i])+','+str(test_ratings[i])+'\n')